# Simple, Interactive, and Dynamic Jeopardy Web App
I recently finished reading Madeleine Albright's engaging *Fascism: A Warning*, and upon completing it two thoughts crossed my mind. First, I was thankful to have access to Alrbight's insights and experiences with world leaders; her unique pedigree and history offers perspective I could not dream of. Second, I was mildly alarmed that for this book—more than most others I have read—I knew I would forget the vast majority of information in the book. Keeping track of Orban, Erdogan, Chavez, and the names of cities, places, and other dictators is just too difficult for me to handle. I could, of course, have kept diligent notes while reading, but I figured that even then I might not be able to recall names and ideas when needed.

Long story much shorter: I want to develop a simple Jeopardy game into which I can feed a keyword and then receive Jeopardy questions related to that keyword. Fortunately, the Jeopardy [Archive](www.j-archive.com) stores hundreds of thousands of past Jeopardy questions, which others have dutifully scraped from the JSON files in the archive.

Because my personal [website](jacobtbigham.com) is hosted on SquareSpace (don't @ me), databasing these questions was a little difficult, since I was forced to use Javascript (and I can't keep Promises, apparently) and could not use SQL. Fortunately, I learned a lot more from coding this project than I bargained for.

A couple of features (and lack of features) worth noting:
- Users input a single keyword or key phrase, to which the app then queries the database for matches.
- Those keywords can match exactly or as substrings within other words (e.g., dog: dog, or erdogan).
- I did not employ any ML algorithms to extract key words, phrases, or topics from questions and answers.
- Instead, I give users the option to either view clues that *only* match the keyword **or** to get clues that matched the keyword *and* the other clues in the same categories as those clues. In this way, I let the Jeopardy writers do my ML for me.
- If there are not enough clues that match a given keyword, then the keyword is rejected.
- I used the Levenshtein distance between users' answers and correct answers to determine whether answers were correct.
- I separated Final Jeopardy and regular Jeopardy questions so that users can get the full Jeopardy experience.
- The first round contains one Daily Double question, and the second round contains two Daily Double questions.
- Users can restart the game with a new keyword at any time.
- Users can share their score to social media sites, along with the keyword for their custom game.
- Users can alternatively opt to play a random Jeopardy game, with clues from random and unrelated categories.

My overall approach was the following:
1. Construct a database to hold regular Jeopardy and Final Jeopardy questions
2. Build a user interface with HTML and Javascript to interact with the database and present questions
3. Enjoy!

Feedback, suggestions, and corrections are greatly appreciated!

## Wrangling

First, let's load the data, which is available [here](https://drive.google.com/drive/folders/1fxY181PdiA1KoJRG23ZVLC2Y5CIRqQWx), and take care of some of the easy wrangling. (I converted from .tsv to .csv format on my computer so that it would open automatically in Excel, wherein I removed the comments and notes sections.)

In [1]:
import pandas as pd
import numpy as np
import unicodedata

In [2]:
data = pd.read_csv("jeopardy.csv", dtype = {"round": np.int16, "value": np.int16})
data.head()

,round,value,daily_double,category,answer,question,air_date
0,1,100,no,LAKES & RIVERS,River mentioned most often in the Bible,the Jordan,9/10/1984
1,1,200,no,LAKES & RIVERS,Scottish word for lake,loch,9/10/1984
2,1,400,no,LAKES & RIVERS,American river only 33 miles shorter than the ...,the Missouri,9/10/1984
3,1,500,no,LAKES & RIVERS,"World's largest lake, nearly 5 times as big as...",the Caspian Sea,9/10/1984
4,1,100,no,INVENTIONS,Marconi's wonderful wireless,the radio,9/10/1984


Let's first verify that the daily_double only has "yes" and "no" values, and then let's change the values to True and False:

In [3]:
data.daily_double.describe()

count     349641
unique         2
top           no
freq      332620
Name: daily_double, dtype: object

In [4]:
data.daily_double.loc[data.daily_double == "no"] = False
data.daily_double.loc[data.daily_double == "yes"] = True
data.head()

C:\Users\Jacob\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,round,value,daily_double,category,answer,question,air_date
0,1,100,False,LAKES & RIVERS,River mentioned most often in the Bible,the Jordan,9/10/1984
1,1,200,False,LAKES & RIVERS,Scottish word for lake,loch,9/10/1984
2,1,400,False,LAKES & RIVERS,American river only 33 miles shorter than the ...,the Missouri,9/10/1984
3,1,500,False,LAKES & RIVERS,"World's largest lake, nearly 5 times as big as...",the Caspian Sea,9/10/1984
4,1,100,False,INVENTIONS,Marconi's wonderful wireless,the radio,9/10/1984


Great! Now, just to be consistent with the capitalization of the categories (and the style of the actual game show answers), let's convert our answers and questions to uppercase:

In [5]:
data["answer"] = data["answer"].str.upper()
data["question"] = data["question"].str.upper()
data.head()

,round,value,daily_double,category,answer,question,air_date
0,1,100,False,LAKES & RIVERS,RIVER MENTIONED MOST OFTEN IN THE BIBLE,THE JORDAN,9/10/1984
1,1,200,False,LAKES & RIVERS,SCOTTISH WORD FOR LAKE,LOCH,9/10/1984
2,1,400,False,LAKES & RIVERS,AMERICAN RIVER ONLY 33 MILES SHORTER THAN THE ...,THE MISSOURI,9/10/1984
3,1,500,False,LAKES & RIVERS,"WORLD'S LARGEST LAKE, NEARLY 5 TIMES AS BIG AS...",THE CASPIAN SEA,9/10/1984
4,1,100,False,INVENTIONS,MARCONI'S WONDERFUL WIRELESS,THE RADIO,9/10/1984


Stellar! Furthermore, though it's not immediately obvious here, many of the catepgories, answers, and questions contain quotation marks that are formatted grotesquely. Let's get those in order:

In [6]:
#for example
data["answer"].iloc[11]

'CLOSE RELATIVE OF THE PIG, THOUGH ITS NAME MEANS "RIVER HORSE\\"'

In [7]:
data["answer"] = data["answer"].str.replace("\\\\", "")
data["question"] = data["question"].str.replace("\\\\", "")
data["category"] = data["category"].str.replace("\\\\", "")
data["answer"].iloc[11]

'CLOSE RELATIVE OF THE PIG, THOUGH ITS NAME MEANS "RIVER HORSE"'

Beautiful! I also want to remove any special/foreign/accented characters (since otherwise a search would not be able to locate them!), so let's do that here using [this](https://stackoverflow.com/a/44433664) function:

In [8]:
#notice the question
data.iloc[16:17]

,round,value,daily_double,category,answer,question,air_date
16,1,400,False,FOREIGN CUISINE,FRENCH FOR A TOOTHSOME CUT OF BEEF SERVED TO A...,CHÂTEAUBRIAND,9/10/1984


In [9]:
def strip_accents(text):
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode("utf-8")
    return str(text)

data["category"] = data["category"].apply(strip_accents)
data["answer"] = data["answer"].apply(strip_accents)
data["question"] = data["question"].apply(strip_accents)

#and now
data.iloc[16:17]

,round,value,daily_double,category,answer,question,air_date
16,1,400,False,FOREIGN CUISINE,FRENCH FOR A TOOTHSOME CUT OF BEEF SERVED TO A...,CHATEAUBRIAND,9/10/1984


Fantastic! Now, because the show double point values on November 26, 2001, we need to adjust clue values prior to that date to match. That the data are arranged chronologically makes this simple:

In [10]:
#notice
data.iloc[129490:129497, ]

,round,value,daily_double,category,answer,question,air_date
129490,2,800,False,"CROSSWORD CLUES ""F""","CYNICAL CINEMA STYLE OF THE '40S (4,4)",FILM NOIR,11/23/2001
129491,2,1000,False,"CROSSWORD CLUES ""F""","BRITISH BAD ""GUY"" (6)",FAWKES,11/23/2001
129492,3,0,False,ANCIENT ROMANS,"ACCORDING TO TRADITION, HE WAS DESCENDED FROM ...",PONTIUS PILATE,11/23/2001
129493,1,200,False,SUDDENLY SUDAN,FROM 1899 TO 1956 SUDAN WAS CONTROLLED BY EGYP...,ENGLAND (GREAT BRITAIN),11/26/2001
129494,1,400,False,SUDDENLY SUDAN,IN 1995 THIS FORMER U.S. PRESIDENT OVERSAW A C...,JIMMY CARTER,11/26/2001
129495,1,600,False,SUDDENLY SUDAN,THE ATBARAH RIVER JOINS THIS RIVER IN NORTHERN...,THE NILE,11/26/2001
129496,1,800,False,SUDDENLY SUDAN,THIS SUDANESE CITY OF ABOUT 1 MILLION WAS FOUN...,KHARTOUM,11/26/2001


In [11]:
data["value"].iloc[:129493] = 2*data["value"].iloc[:129493]
data.iloc[129490:129497, ]

,round,value,daily_double,category,answer,question,air_date
129490,2,1600,False,"CROSSWORD CLUES ""F""","CYNICAL CINEMA STYLE OF THE '40S (4,4)",FILM NOIR,11/23/2001
129491,2,2000,False,"CROSSWORD CLUES ""F""","BRITISH BAD ""GUY"" (6)",FAWKES,11/23/2001
129492,3,0,False,ANCIENT ROMANS,"ACCORDING TO TRADITION, HE WAS DESCENDED FROM ...",PONTIUS PILATE,11/23/2001
129493,1,200,False,SUDDENLY SUDAN,FROM 1899 TO 1956 SUDAN WAS CONTROLLED BY EGYP...,ENGLAND (GREAT BRITAIN),11/26/2001
129494,1,400,False,SUDDENLY SUDAN,IN 1995 THIS FORMER U.S. PRESIDENT OVERSAW A C...,JIMMY CARTER,11/26/2001
129495,1,600,False,SUDDENLY SUDAN,THE ATBARAH RIVER JOINS THIS RIVER IN NORTHERN...,THE NILE,11/26/2001
129496,1,800,False,SUDDENLY SUDAN,THIS SUDANESE CITY OF ABOUT 1 MILLION WAS FOUN...,KHARTOUM,11/26/2001


Miraculous! By the way, notice that Final Jeopardy questions are indicated by a round value of 3, and their corresponding value is 0. Now, also not obvious is that not every point value is correct or possible:

In [12]:
ACCEPTABLE_VALUES = [200, 400, 600, 800, 1000, 1200, 1600, 2000]

data.loc[(data["round"] != 3) & ((~data["value"].isin(ACCEPTABLE_VALUES))|data["daily_double"])]

,round,value,daily_double,category,answer,question,air_date
44,2,2000,True,4-LETTER WORDS,"IT'S THE FIRST 4-LETTER WORD IN ""THE STAR SPAN...",WHAT,9/10/1984
69,1,800,True,BY THE NUMBERS,TOTAL OF DISNEY'S DALMATIANS AND DWARFS,108,9/11/1984
107,1,800,True,AUTOMOBILES,"MODELS OF FORD, CHRYSLER, & CADILLAC ARE ALL N...",SPAIN,9/12/1984
126,2,3000,True,NEW TESTAMENT,"""BLESSED ARE"" THESE ""FOR THEY SHALL BE CALLED ...",THE PEACEMAKERS,9/12/1984
179,2,7000,True,ANATOMY,"PROMINENT ZYGOMATIC BONES, CONSIDERED AN ATTRA...",CHEEK BONES,9/13/1984
...,...,...,...,...,...,...,...
349545,1,1000,True,TV SPINOFFS,"""TORCHWOOD"" WAS A SPINOFF OF THIS BRITISH SERI...",DOCTOR WHO,7/25/2019
349566,2,4000,True,SECRETARIES OF WAR,IN EARLY 1801 SEC. OF WAR SAMUEL DEXTER FOUND ...,JOHN ADAMS,7/25/2019
349570,2,4000,True,GATES,"THE NAME OF THIS ANCIENT CITY, HOME TO THE ISH...",BABYLON,7/25/2019
349605,1,1000,True,BEFORE & AFTER: SINGERS & WRITERS,"""HIT THE ROAD JACK"" WITH THE SINGER & PIANIST ...",RAY CHARLES DARWIN,7/26/2019


In [13]:
data.loc[349562:349571, ]

,round,value,daily_double,category,answer,question,air_date
349562,1,600,False,FOR ALL IT'S WORTH,THE ORG. JEWELERS OF AMERICA SAYS CONTRARY TO ...,A DIAMOND,7/25/2019
349563,1,800,False,FOR ALL IT'S WORTH,APPRAISERS ON THIS PBS SHOW OFTEN GIVE A PIECE...,ANTIQUES ROADSHOW,7/25/2019
349564,1,1000,False,FOR ALL IT'S WORTH,THIS VENERABLE LONDON GALLERY LOST 2 TURNERS T...,THE TATE,7/25/2019
349565,2,800,False,SECRETARIES OF WAR,"LAST NAME OF ALPHONSO & SON WILLIAM HOWARD, BO...",TAFT,7/25/2019
349566,2,4000,True,SECRETARIES OF WAR,IN EARLY 1801 SEC. OF WAR SAMUEL DEXTER FOUND ...,JOHN ADAMS,7/25/2019
349567,2,1600,False,SECRETARIES OF WAR,THIS CHRISTMAS PLANT WITH RED LEAVES WAS NAMED...,A POINSETTIA,7/25/2019
349568,2,2000,False,SECRETARIES OF WAR,"HOW AWKWARD! JOHN FLOYD, THIS PRESIDENT'S SECR...",BUCHANAN,7/25/2019
349569,2,400,False,GATES,THIS ALLITERATIVE STRAIT WAS ONCE CALLED BOCA ...,GOLDEN GATE,7/25/2019
349570,2,4000,True,GATES,"THE NAME OF THIS ANCIENT CITY, HOME TO THE ISH...",BABYLON,7/25/2019
349571,2,1600,False,GATES,"HELL GATE, FROM A DUTCH WORD, CONNECTS THIS ""D...",THE EAST RIVER,7/25/2019


As if by providence, we notice here another problem as well: not all the clues are here, but not for the reason we might first suppose (that the clue was never revealed). If we look at the Jeopardy archive [page](http://www.j-archive.com/showgame.php?game_id=6388) for 7-25-2019, the reasons for these problems become clear..

The 4000-point clues, for example, are Daily Doubles! And the missing clue in the "GATES" category is a clue that contained an image—and those clues are omitted from the dataset.

I'm going to impute the Daily Double values by assigning them whatever value is missing based on the other clues for each category. Notably, sometimes there are missing clues in categories with Daily Doubles. In such cases, I will assign to the Daily Double question the highest of missing values, since, in the past, Daily Double questions tended lower on the board.

It's mildly more costly but easier to implement this by running over the entire dataset, not just the problem-values we identified before. This approach makes is easier to identify category chunks. I could have used a swifter Pandas aggregate and sort approach, but the iterative approach below works and isn't so slow that it's worth scrapping.

In [14]:
indexes_with_problems = [index for index in range(0, len(data)) if data["round"].iloc[index] != 3 \
                                                                and ((data["value"].iloc[index] not in ACCEPTABLE_VALUES)
                                                                     or data["daily_double"].iloc[index])]
len(indexes_with_problems)
assert(len(indexes_with_problems) == 17025)

In [15]:
ROUND_ONE_VALUES = set([200, 400, 600,  800,  1000])
ROUND_TWO_VALUES = set([400, 800, 1200, 1600, 2000])

index = 0
while(index < 349630):        #the last problem is at 349625
    current_category = data["category"].iloc[index]
    indexes_in_category = [index]
    index += 1
    while(data["category"].iloc[index] == current_category):
        indexes_in_category.append(index)
        index += 1
    for i in indexes_in_category:
        if i in indexes_with_problems:
            indexes_in_category.remove(i)
            possible_values = ROUND_ONE_VALUES.copy() if data["round"].iloc[i] == 1 else ROUND_TWO_VALUES.copy()
            for j in indexes_in_category:
                if data["value"].iloc[j] in possible_values:
                    possible_values.remove(data["value"].iloc[j])
                else:
                    print("Problem removing value from line", j, end= "\n")
            data["value"].iloc[i] = max(possible_values)
            break

Problem removing value from line 4257
Problem removing value from line 77348
Problem removing value from line 79593


Interesting problems were at fault here! For element 4257, which was a Final Jeopardy question, its category—EUROPE—was the same as the first category for the next show, so the code saw them all as one category. There's no need for an edit here, but I show the cells below to verify. The other two problematic values occurred for episodes that contained "bonus" rounds, where contestants could give either one or two of the two possible answers. I'll edit these manually since there are only two of these rounds:

In [16]:
data.iloc[4257:4263]

,round,value,daily_double,category,answer,question,air_date
4257,3,0,False,EUROPE,ALL OF DENMARK IS SURROUNDED BY WATER EXCEPT F...,GERMANY,2/11/1986
4258,1,200,False,EUROPE,THIS NATION'S OFFICIAL NAME IS THE HELLENIC RE...,GREECE,2/12/1986
4259,1,400,False,EUROPE,"CITIZEN'S OF THIS SMALL COUNTRY AREN'T ""NEUTRA...",SWITZERLAND,2/12/1986
4260,1,600,False,EUROPE,THIS COUNTRY'S CITY OF SHEFFIELD HAS BEEN AN I...,ENGLAND,2/12/1986
4261,1,800,False,EUROPE,THIS COUNTRY THAT HAS MANY SATELLITES BUILT A ...,RUSSIA,2/12/1986
4262,1,1000,True,EUROPE,REAL BOHEMIANS AREN'T ARTISTS BUT RESIDENTS OF...,CZECHOSLOVAKIA,2/12/1986


In [17]:
data.iloc[77347:77352]

,round,value,daily_double,category,answer,question,air_date
77347,2,800,False,BONUS MOVIE,"HE PLAYED MR. BANKS, THE ""FATHER OF THE BRIDE""",STEVE MARTIN & SPENCER TRACY,12/16/1997
77348,2,800,False,BONUS MOVIE,DAN AYKROYD STARRED IN THIS FILM BASED ON CHAR...,CONEHEADS & THE BLUES BROTHERS,12/16/1997
77349,2,1200,False,BONUS MOVIE,LATE '80S KEVIN COSTNER FILM WHOSE SUBJECT WAS...,BULL DURHAM & FIELD OF DREAMS,12/16/1997
77350,2,1600,False,BONUS MOVIE,SHE WON THE 1968 ACADEMY AWARD FOR BEST ACTRESS,KATHARINE HEPBURN & BARBRA STREISAND,12/16/1997
77351,2,2000,False,BONUS MOVIE,"WOODY ALLEN FILM WITH ""SEX"" IN THE TITLE",A MIDSUMMER NIGHT'S SEX COMEDY & EVERYTHING YO...,12/16/1997


In [18]:
data["value"].iloc[77347] = 400

In [19]:
data.iloc[79592:79597]

,round,value,daily_double,category,answer,question,air_date
79592,2,800,False,TOP 40 BONUS,"""PROUD MARY""",CREEDENCE CLEARWATER REVIVAL & IKE & TINA TURNER,2/10/1998
79593,2,800,False,TOP 40 BONUS,"""LIVE AND LET DIE""",WINGS & GUNS N' ROSES,2/10/1998
79594,2,1200,False,TOP 40 BONUS,"""CALIFORNIA GIRLS""",THE BEACH BOYS & DAVID LEE ROTH,2/10/1998
79595,2,1600,False,TOP 40 BONUS,"""BLUE BAYOU""",LINDA RONSTADT & ROY ORBISON,2/10/1998
79596,2,2000,False,TOP 40 BONUS,"""SOUL MAN""",THE BLUES BROTHERS & SAM & DAVE,2/10/1998


In [20]:
data["value"].iloc[79592] = 400

Now, just as a final sanity check, let's make sure that every category has *unique* point values and adjust any that don't:

In [21]:
data.tail(6)

,round,value,daily_double,category,answer,question,air_date
349635,2,2000,False,SHAKESPEAREAN DOUBLE BLANK VERSE,"A DOUBLE DOUBLE BLANK, FROM ""MACBETH"": ""____ I...","""FAIR IS FOUL, AND FOUL IS FAIR""",7/26/2019
349636,2,400,False,MAKE IT SNAPPY,"AS WELL AS PHOTOSHARING ON THIS APP, YOU CAN W...",SNAPCHAT,7/26/2019
349637,2,800,False,MAKE IT SNAPPY,"GENUS ANTIRRHINUM, THESE FLOWERS SNAP CLOSED A...",SNAPDRAGONS,7/26/2019
349638,2,1600,False,MAKE IT SNAPPY,THIS HYPHENATED TOOL COMPANY OWNS BRANDS LIKE ...,SNAP-ON,7/26/2019
349639,2,2000,False,MAKE IT SNAPPY,"IN 2019 METEOROLOGIST DARYL RITCHISON AT NDSU,...",NORTH DAKOTA STATE UNIVERSITY,7/26/2019
349640,3,0,False,HISTORIC SHIPS,215 PASSENGERS WERE RESCUED WHEN IT SANK IN JU...,THE CARPATHIA,7/26/2019


In [22]:
index = 0
while(index < 349636):        #since we see above there is no problem with the last round
    while data["round"].iloc[index] == 3: #some Final Jeopardy questions appear successively without any round content
        index += 1
    current_category = data["category"].iloc[index]
    indexes_in_category = [index]
    index += 1
    while(data["category"].iloc[index] == current_category):
        indexes_in_category.append(index)
        index += 1
    for i in indexes_in_category:
        possible_values = ROUND_ONE_VALUES.copy() if data["round"].iloc[i] == 1 else ROUND_TWO_VALUES.copy()
        for j in indexes_in_category:
            if data["value"].iloc[j] in possible_values:
                possible_values.remove(data["value"].iloc[j])
            else:
                print("Problem removing value from line ", j, end= "\n")
                break

Problem removing value from line  44066
Problem removing value from line  44066
Problem removing value from line  44066


In [23]:
data.iloc[44064:44067]

,round,value,daily_double,category,answer,question,air_date
44064,1,200,False,"DON'T ""CALL"" US","1903 JACK LONDON WORK IN WHICH A DOG, HALF ST....",THE CALL OF THE WILD,10/30/1992
44065,1,600,False,"DON'T ""CALL"" US",IT'S THE APPEARANCE OF A PERFORMER AFTER THE P...,A CURTAIN CALL,10/30/1992
44066,1,600,False,"DON'T ""CALL"" US","""BOVINE"" TERM FOR AN AUDITION OPEN TO A LARGE ...",A CATTLE CALL,10/30/1992


This is simply a typo in the J-Archive data, and I will fix it:

In [24]:
data["value"].iloc[44065] = 400
data.iloc[44064:44067]

,round,value,daily_double,category,answer,question,air_date
44064,1,200,False,"DON'T ""CALL"" US","1903 JACK LONDON WORK IN WHICH A DOG, HALF ST....",THE CALL OF THE WILD,10/30/1992
44065,1,400,False,"DON'T ""CALL"" US",IT'S THE APPEARANCE OF A PERFORMER AFTER THE P...,A CURTAIN CALL,10/30/1992
44066,1,600,False,"DON'T ""CALL"" US","""BOVINE"" TERM FOR AN AUDITION OPEN TO A LARGE ...",A CATTLE CALL,10/30/1992


All set! There are some further changes I will make to the data for reasons idiosyncratic to my use case, but I'm exporting this set as `jeopardy_clean.csv` for anyone else to use. I think many would find the clean-ups beneficial. (Plus, I'll likely need to come back to this as a starting point for any changes, and I don't want to rerun the above code every time!)

In [25]:
data.to_csv("jeopardy_clean.csv", index = False)

## App-Specific Wrangling
Thinking ahead about implementation, I'd like to make a few little fixes. 

First, since I really don't care about the exact air date of each question, but I *do* care about the relative air dates (I'd like to prioritize newer questions), **I'm going to remove the air_date column**. The relative order of air dates is contained in the DataFrame index. Indeed, since database queries will access questions from top to bottom, **I'm going to reverse the vertical order of the entries**—with the caveat (CC) introduced below.

Second, I really don't care whether a clue was a Daily Double question, for two reasons: one, since I'm going to be pulling from questions across many categories anyways, it might be disruptive and unnecessarily costly to preserve this data; and two, and more importantly, the game has changed such that the location and relative difficulty of Daily Double questions seems random. So, **I'm going to drop the daily-double column**.

Third, I don't so much care what round in which a question appeared. I do, however, care whether it was a Final Jeopardy question, since the syntax of those questions (and their difficulty) tends to differ from those of the normal rounds. I could add a separate column to track this—and remove the round column—but what **I'm going to instead do is create a separate DataFrame with just Final Jeopardy questions**. My rationale for this is speed: I'll only want to search for Final Jeopardy questions *when it's time for Final Jeopardy*, so there's no sense in keeping an unneeded extra column (round) or in keeping unneeded extra clues (the Final Jeopardy clues) in the same data structure as the normal clues. **I'll remove the round column after that separation**.

Fourth, there are some clues that originally had audio and video accompaniments that are not replicated here. Often these clues contain the phrase "heard here" or "seen here." **I will remove clues that contain these phrases.**

Finally, I also don't *really* care how much each question is worth; I only care about the relative order, which is contained in the index. So, **I'm going to drop the value column**. I do care, however, whether questions were in the same category group. Because category names are not unique, **I'm going to give each category from each game a distinct category_id**. (CC) Because I'll be flipping the vertical order of the clues, **I'll first flip the vertical order of the clues in each category**, such that after overall flipping, the harder (higher-valued) questions appear lower in the database.

So, let's go ahead and make these final changes:
1. Remove the daily_double column
2. Give each category a unique category_id
3. Flip the vertical order of questions in each category
4. Remove the value column
5. Remove the air_date column
6. Flip the overall vertical order of clues
7. Remove and clues with "heard here" or "seen here" in the answer
8. Separate regular and Final Jeopardy questions
9. Remove the round column

In [53]:
data = pd.read_csv("jeopardy_clean.csv")

In [54]:
del data["daily_double"] #1

data["air_date"] = pd.to_datetime(data["air_date"], format = "%m/%d/%Y") #so that ids are in order by date

data["category_id"] = data.groupby(["air_date", "category"]).ngroup() + 1 #2

data.sort_values(by=["category_id", "value"], ascending=[False, False], inplace = True) #3 and #6

del data["value"] #4
del data["air_date"] #5

bad_strings = ["HEARD HERE", "SEEN HERE", "SHOWN HERE", "PICTURED HERE"]
for string in bad_strings:
    data = data[~data["answer"].str.contains(string)] #7

normal_questions = data.loc[data["round"] != 3] #8
final_questions = data.loc[data["round"] == 3]

del normal_questions["round"] #9
del final_questions["round"]

normal_questions.reset_index(inplace = True, drop = True)
final_questions.reset_index(inplace = True, drop = True)

normal_questions.head(8)

,category,answer,question,category_id
0,VENICE,IN 697 PAOLO LUCIO ANAFESTO BECAME THE FIRST O...,DOGE,79826
1,VENICE,"DURING ONE SEASON, THE WINTER O 1678, THIS BAR...",ANTONIO VIVALDI,79826
2,VENICE,"NOW MEANING A SLUM, THIS WORD MAY COME FROM TH...",GHETTO,79826
3,TURNING TO SPORTS,"FOX SPORTS: THIS UTAH JAZZ MAN, AKA THE MAILMA...",KARL MALONE,79825
4,TURNING TO SPORTS,"THIS ""BROWN BOMBER"" SUCCESSFULLY DEFENDED HIS ...",JOE LOUIS,79825
5,TURNING TO SPORTS,"SPIN IN AN 8-FOOT, 2-INCH THROWING CIRCLE TO T...",DISCUS,79825
6,TURNING TO SPORTS,LEE CORSO ONCE NOTED THE RAINBOW WARRIORS FOOT...,(THE UNIVERSITY OF) HAWAII,79825
7,TURNING TO SPORTS,IN 201 A U.K. GAME OF THIS SPORT WENT 50 HOURS...,CRICKET,79825


In [55]:
final_questions.head(5)

,category,answer,question,category_id
0,HISTORIC SHIPS,215 PASSENGERS WERE RESCUED WHEN IT SANK IN JU...,THE CARPATHIA,79819
1,1970s ALBUM REVIEWS,"ROLLING STONE SAID THIS 1976 ALBUM HAD ""THE BE...",HOTEL CALIFORNIA,79801
2,CHILDREN'S AUTHORS,THIS AUTHOR & ILLUSTRATOR WHO WON THE 1964 CAL...,MAURICE SENDAK,79791
3,TOYS & GAMES,THE PROTOTYPE FOR THIS GAME THAT WAS INTRODUCE...,SCRABBLE,79787
4,LANDMARKS,DAVID LIVINGSTONE WROTE OF THIS DISCOVERY OF H...,VICTORIA FALLS,79770


So that I need not run this processing code every time I want to use the cleaned data, let's go ahead and save these DataFrames in new .csv files called `jeopardy_normal.csv` and `jeopardy_final.csv`.

In [56]:
normal_questions.to_csv("jeopardy_normal.csv", index = False)
final_questions.to_csv("jeopardy_final.csv", index = False)

In [57]:
normals = pd.read_csv("jeopardy_normal.csv")
finals = pd.read_csv("jeopardy_final.csv")

Finally, let's see how many total clues, categories, and Final Jeopardy questions we have in this set:

In [58]:
num_categories = len(normals["category_id"].unique())
num_finals = len(finals)
num_clues = len(normals)
print("There are {:,} clues across {:,} categories and {:,} Final Jeopardy questions"\
          .format(num_clues,        num_categories,     num_finals), end=".")

There are 341,189 clues across 73,588 categories and 6,196 Final Jeopardy questions.

## Preparation for Database Querying
The fact that I have separate columns for category, answer, and question is not especially memory-intensive. Namely, I have to separatet the values *somehow*, and it's no costlier to store a comma than it is to store any other character. However, when I run database queries, it *is* costlier and requires unnecessarily more lines of code to run three queries (on category, answer, and question) and then merge the results than it is to just run one query for matches on the aggregate of those three values. So, I'm going to aggregate the data into just one block of text, where the category, question, and answer are separated by a `|` symbol.

In [59]:
finals["agg"] = finals["category"] + "|" + finals["answer"] + "|" + finals["question"]
del finals["category"]
del finals["answer"]
del finals["question"]
finals.head()

,category_id,agg
0,79819,HISTORIC SHIPS|215 PASSENGERS WERE RESCUED WHE...
1,79801,1970s ALBUM REVIEWS|ROLLING STONE SAID THIS 19...
2,79791,CHILDREN'S AUTHORS|THIS AUTHOR & ILLUSTRATOR W...
3,79787,TOYS & GAMES|THE PROTOTYPE FOR THIS GAME THAT ...
4,79770,LANDMARKS|DAVID LIVINGSTONE WROTE OF THIS DISC...


In [60]:
finals.to_csv("jeopardy_final_agg.csv", index = False)

In [61]:
normals["agg"] = normals["category"] + "|" + normals["answer"] + "|" + normals["question"]
del normals["category"]
del normals["answer"]
del normals["question"]
normals.head()

,category_id,agg
0,79826,VENICE|IN 697 PAOLO LUCIO ANAFESTO BECAME THE ...
1,79826,"VENICE|DURING ONE SEASON, THE WINTER O 1678, T..."
2,79826,"VENICE|NOW MEANING A SLUM, THIS WORD MAY COME ..."
3,79825,TURNING TO SPORTS|FOX SPORTS: THIS UTAH JAZZ M...
4,79825,"TURNING TO SPORTS|THIS ""BROWN BOMBER"" SUCCESSF..."


First iterations of my implementation of the game allowed for the board to have fewer than 25 clues. For example, in the VENICE category in the above output, on the show in which that category was played, only three of the clues were revealed. I figured it would be better (and more inclusive) to give those categories a chance, but it just looks gross, especially when there are only 1 or 2 clues in a category. So, I'm going to assign any clues that belong to an incomplete category a category_id of 0, which I can then exclude from database searches when desired. Notably, this still allows clues from incomplete categories to appear in games in which the user does not opt to keep categories intact.

In [62]:
normals["freq"] = normals.groupby("category_id")["category_id"].transform("count")
normals["category_id"].loc[normals["freq"] < 5] = 0
del normals["freq"]
normals.head()

,category_id,agg
0,0,VENICE|IN 697 PAOLO LUCIO ANAFESTO BECAME THE ...
1,0,"VENICE|DURING ONE SEASON, THE WINTER O 1678, T..."
2,0,"VENICE|NOW MEANING A SLUM, THIS WORD MAY COME ..."
3,79825,TURNING TO SPORTS|FOX SPORTS: THIS UTAH JAZZ M...
4,79825,"TURNING TO SPORTS|THIS ""BROWN BOMBER"" SUCCESSF..."


Because there are limitations on file size for uploading to the database on Back4All, I'm going to split this set into 4 chunks. Whether the chunks split cleanly between categories is not relevant, since they'll be reaggregated in the database. New imports stack atop old imports, so chunk 1 will be the bottom quarter, up to chunk 4 the top quarter.

Notably, I could always just use a sort on the category_ids, but I'm not super sure how efficient the database I'm using is!

In [63]:
size = len(normals)
for quarter in range(1, 5):
    chunk = normals.iloc[int((quarter-1)*size/4) : int(quarter*size/4), ]
    chunk.to_csv("jeopardy_normal_agg" + str(5-quarter) + ".csv", index = False)

That does it for the preparatory steps. Onward we forge to the implementation!